In [1]:
from pathlib import Path
from qa2nli.qa_readers import race 
import tqdm
from typing import *
from joblib import delayed, Parallel
import json

In [2]:
_set = 'train'
nli_data = Path('.data/RACE/set1/nli_{}.json'.format(_set))
output_path = Path('.data/RACE/set1/nli_in_qa_format_{}.json'.format(_set))

In [3]:
#load converted nli data
with open(nli_data) as f:
    nli_samples = json.load(f)

In [4]:
nli_in_qa_format = {}
for nli_sample in nli_samples:
    filename, q_number, opt = nli_sample['id'].split('_')
    q_id = ('_').join([filename, q_number])
    qa_format_sample = nli_in_qa_format.get(q_id, {'id': q_id})
    if len(qa_format_sample) ==1:
        qa_format_sample['premise'] = nli_sample['premise']
        qa_format_sample['hypothesis_options'] = [nli_sample['hypothesis']]
        qa_format_sample['one_hot_label'] = [int(nli_sample['label'])]
        nli_in_qa_format[q_id] = qa_format_sample
    else:
        qa_format_sample['hypothesis_options'].append(nli_sample['hypothesis'])
        qa_format_sample['one_hot_label'].append(int(nli_sample['label']))
    
    

In [5]:
len(nli_in_qa_format)

44890

In [6]:
len(nli_samples)//4

44890

In [7]:
# add another label field
for idx, sample in nli_in_qa_format.items():
    sample['label'] = sample['one_hot_label'].index(1)

In [8]:
with open(output_path, 'w') as f:
    json.dump(list(nli_in_qa_format.values()), f)